# 1- Import Libraries

In [1]:
import pandas as pd
import numpy as np
import requests
import re
from io import BytesIO
from zipfile import ZipFile
from sklearn.feature_extraction.text import CountVectorizer
from urllib.parse import urlparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

##1.1- Download the dataset

In [2]:
url ='https://github.com/psonkar07/Job-Portal-Analysis/raw/main/jobs.csv.zip'
response = requests.get(url)

if response.status_code == 200:
    zip_file = BytesIO(response.content)
    with ZipFile(zip_file) as z:
        print(z.namelist())
        with z.open('jobs.csv') as f:
            df = pd.read_csv(f)
            print(df.head())
else:
    print(f"Failed to download file: {response.status_code}")



['jobs.csv']
         job_id                                           job_role    company  \
0  7.012301e+10      Branch Banking - Calling For Women Candidates  Hdfc Bank   
1  6.012391e+10                       Product Owner Senior Manager  Accenture   
2  6.012391e+10  Employee Relations and Policies Associate Manager  Accenture   
3  6.012391e+10         Employee Relations and Policies Specialist  Accenture   
4  6.012301e+10                                  SAP BO Consultant   Mindtree   

  experience         salary  \
0    1-6 Yrs  Not disclosed   
1  11-15 Yrs  Not disclosed   
2    3-7 Yrs  Not disclosed   
3    3-7 Yrs  Not disclosed   
4    5-7 Yrs  Not disclosed   

                                            location  rating        reviews  \
0  Kolkata, Hyderabad/Secunderabad, Pune, Ahmedab...     4.0  39110 Reviews   
1  Kolkata, Mumbai, Hyderabad/Secunderabad, Pune,...     4.1  32129 Reviews   
2  Kolkata, Mumbai, Hyderabad/Secunderabad, Pune,...     4.1  32129 Reviews 

## 1.2- Dataset info

In [3]:
print("\nDataset summary:")
print(df.info())


Dataset summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79147 entries, 0 to 79146
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   job_id           78667 non-null  float64
 1   job_role         78667 non-null  object 
 2   company          78666 non-null  object 
 3   experience       77398 non-null  object 
 4   salary           78667 non-null  object 
 5   location         77441 non-null  object 
 6   rating           42948 non-null  float64
 7   reviews          42948 non-null  object 
 8   resposibilities  78647 non-null  object 
 9   posted_on        78667 non-null  object 
 10  job_link         78667 non-null  object 
 11  company_link     78667 non-null  object 
dtypes: float64(2), object(10)
memory usage: 7.2+ MB
None


## 1.3- Checking Null Values

In [4]:
df.isnull().sum()

,0
job_id,480
job_role,480
company,481
experience,1749
salary,480
location,1706
rating,36199
reviews,36199
resposibilities,500
posted_on,480


#2- Data Cleaning

In [5]:
# Correct column name
df.rename(columns={'resposibilities': 'responsibilities'}, inplace=True)

# Handling null values
df.dropna(subset=['job_id', 'job_role', 'company', 'experience', 'salary', 'location','responsibilities', 'posted_on', 'job_link','company_link'], inplace=True)
df['experience'].fillna('Unknown', inplace=True)
df['location'].fillna('Unknown', inplace=True)
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

# Convert reviews to string first
df['reviews'] = df['reviews'].astype(str).str.replace(' Reviews', '').astype(float, errors='ignore')

median_rating = df['rating'].median()
median_reviews = df['reviews'].median()
df['rating'].fillna(median_rating, inplace=True)
df['reviews'].fillna(median_reviews, inplace=True)
df['responsibilities'].fillna('Not specified', inplace=True)

# Correcting data types
df['job_id'] = df['job_id'].astype(str)
df['experience'] = df['experience'].str.strip()
df['salary'] = df['salary'].str.strip()
df['location'] = df['location'].str.strip()
df = df.drop_duplicates(subset = 'job_link')
print(df.isnull().sum())

job_id              0
job_role            0
company             0
experience          0
salary              0
location            0
rating              0
reviews             0
responsibilities    0
posted_on           0
job_link            0
company_link        0
dtype: int64


#3- Exploratory Data Analysis(EDA)
###3.1- Exploring job_id column

In [6]:
# Check for uniquw numbers of 'job_id'column
unique_ids = df['job_id'].nunique()
print(f"Unique job_id entries: {unique_ids}")

#convert into string
df['job_id'] = df['job_id'].astype(str)

# Check for duplicates in 'job_id' column
duplicate_ids = df['job_id'].duplicated().sum()
print(f"Number of duplicate job_ids: {duplicate_ids}")

# Check for null values in 'job_id' column
missing_job_ids = df['job_id'].isnull().sum()
print(f"Number of missing job_ids: {missing_job_ids}")

# After the changes
print("\nAfter cleaning:")
print(f"Total job_id entries: {df['job_id'].count()}")
print(f"Unique job_id entries: {df['job_id'].nunique()}")
print(f"Number of duplicate job_ids: {df['job_id'].duplicated().sum()}")
print(f"Number of missing job_ids: {df['job_id'].isnull().sum()}")

Unique job_id entries: 71738
Number of duplicate job_ids: 0
Number of missing job_ids: 0

After cleaning:
Total job_id entries: 71738
Unique job_id entries: 71738
Number of duplicate job_ids: 0
Number of missing job_ids: 0


### 3.2- Exploring job_role column

In [7]:
# Check for unique job roles
print(f"Number of unique job roles: {df['job_role'].nunique()}")

# Distribution of job roles
print("\nTop 10 job roles:")
print(df['job_role'].value_counts().head(10))

# Standardize job role column
df['job_role'] = df['job_role'].str.strip().str.title()


Number of unique job roles: 50240

Top 10 job roles:
job_role
Business Development Executive    296
Sales Executive                   261
Java Developer                    249
Business Development Manager      238
Sales Manager                     153
Senior Software Engineer          143
Graphic Designer                  141
Business Analyst                  136
Software Engineer                 131
Dot Net Developer                 130
Name: count, dtype: int64


### 3.3- Exploring company column

In [8]:
# Checking for unique companies
unique_companies = df['company'].nunique()
print(f"Number of unique companies: {unique_companies}")

# Distribution of companies
company_counts = df['company'].value_counts().head(10)
print(f"Frequency of each company: {company_counts}")

# Handle null Values
missing_companies = df['company'].isnull().sum()
print(f"Number of missing companies: {missing_companies}")

# Remove whitespaces and standardize text
df['company'] = df['company'].str.strip().str.title()


# After the changes
print("\nAfter cleaning:")
print(f"Number of unique companies after cleaning: {df['company'].nunique()}")
print(df['company'].value_counts().head(10))
print(f"Number of missing companies after cleaning: {df['company'].isnull().sum()}")

Number of unique companies: 14927
Frequency of each company: company
Lavya Associates          5128
Accenture                 2928
Hucon                      852
Varite India Pvt. Ltd.     815
IBM                        656
Megma Services             523
Infosys                    505
D'source                   502
RCPC                       444
Tekpillar Services         377
Name: count, dtype: int64
Number of missing companies: 0

After cleaning:
Number of unique companies after cleaning: 14777
company
Lavya Associates          5128
Accenture                 2928
Hucon                      852
Varite India Pvt. Ltd.     815
Ibm                        656
Megma Services             523
Infosys                    505
D'Source                   502
Rcpc                       444
Tekpillar Services         377
Name: count, dtype: int64
Number of missing companies after cleaning: 0


###3.4- Exploring experience column

In [9]:
# Remove whitespaces and standardize text
df['experience'] = df['experience'].str.strip()

# Handle null Values
missing_experience = df['experience'].isnull().sum()
print(f"Number of missing experience values: {missing_experience}")

#Categorize experience
def experience_to_range(exp):
    if '-' in exp:
        min_exp, max_exp = map(int, exp.replace(' Yrs', '').split('-'))
        return (min_exp + max_exp) / 2
    elif 'Yrs' in exp:
        return int(exp.split()[0])
    elif exp == 'Unknown':              # Handle the 'Unknown' case directly
        return 0
    else:
        return None

#Apply the function to categorize experience
df['experience_range'] = df['experience'].apply(experience_to_range)

# After the changes
print("\nAfter cleaning:")
print(df['experience_range'].value_counts())
print(f"Number of unique experience categories: {df['experience_range'].nunique()}")
print(f"Number of missing experience values after cleaning: {df['experience'].isnull().sum()}")


Number of missing experience values: 0

After cleaning:
experience_range
7.5     7849
3.5     6247
2.5     5572
5.5     5073
4.5     5010
6.5     4421
3.0     4256
4.0     3814
5.0     3755
2.0     3701
1.5     2771
6.0     2459
0.5     1989
1.0     1952
7.0     1566
8.5     1535
9.5     1264
10.5    1132
9.0      990
8.0      901
12.5     819
15.0     706
10.0     623
0.0      541
11.0     421
13.5     357
11.5     342
17.5     280
14.0     195
12.0     169
16.0     147
13.0     142
19.5     112
14.5      93
17.0      85
16.5      78
15.5      66
19.0      45
18.5      34
22.5      31
18.0      30
25.0      23
21.5      23
20.0      19
20.5      18
21.0      14
22.0      11
23.0       9
24.0       9
28.0       8
23.5       7
24.5       5
25.5       5
27.5       4
28.5       4
26.5       4
26.0       2
Name: count, dtype: int64
Number of unique experience categories: 57
Number of missing experience values after cleaning: 0


### 3.5- Exploring salary column


In [10]:
# Remove white spaces
df['salary'] = df['salary'].str.strip()

# Handle null values
missing_salaries = df['salary'].isnull().sum()
print(f"Number of missing salary values: {missing_salaries}")

if missing_salaries > 0:
    df['salary'].fillna('Not Disclosed', inplace=True)
    print("Missing salary values filled with 'Not Disclosed'.")

# clean salary ranges
def extract_average_salary(salary):
    if salary == "Not Disclosed":
        return salary
    else:
        match = re.search(r'([\d,]+)\s*-\s*([\d,]+)\s*PA', salary)
        if match:
            lower_salary = int(match.group(1).replace(',', ''))
            upper_salary = int(match.group(2).replace(',', ''))
            return (lower_salary + upper_salary) / 2
        return salary

df['salary_range'] = df['salary'].apply(extract_average_salary)

# After the changes
print("\nAfter cleaning:")
print(df[['salary', 'salary_range']])


Number of missing salary values: 0

After cleaning:
                        salary   salary_range
0                Not disclosed  Not disclosed
1                Not disclosed  Not disclosed
2                Not disclosed  Not disclosed
3                Not disclosed  Not disclosed
4                Not disclosed  Not disclosed
...                        ...            ...
73762  7,00,000 - 8,50,000 PA.       775000.0
73763  7,00,000 - 8,50,000 PA.       775000.0
73764  3,00,000 - 4,50,000 PA.       375000.0
73765            Not disclosed  Not disclosed
73766            Not disclosed  Not disclosed

[71738 rows x 2 columns]


###3.6- Exploring location column

In [11]:
# Data Preprocessing
df['location'] = df['location'].str.strip().str.title().fillna('Unknown')

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['location'])

# Clustering
num_clusters = 10  # Adjust the number of clusters as needed
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df['location_cluster'] = kmeans.fit_predict(X)

# Automate Replacement Based on Clusters
cluster_replacements = {}
for cluster_num in range(num_clusters):
    cluster_locations = df[df['location_cluster'] == cluster_num]['location']
    most_common_term = cluster_locations.value_counts().idxmax()
    for location in cluster_locations.unique():
        cluster_replacements[location] = most_common_term

# Replace Synonyms
df['location'] = df['location'].replace(cluster_replacements)

# Verify the Changes
print("\nVerification after cleaning:")
print(df['location'].value_counts().head(10))
print(f"Number of unique locations after cleaning: {df['location'].nunique()}")
print(f"Number of missing locations after cleaning: {df['location'].isnull().sum()}")

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)



Verification after cleaning:
location
Permanent Remote          28486
Bangalore/Bengaluru       10862
Hyderabad/Secunderabad     6597
Mumbai                     5643
Delhi / Ncr                4815
Gurgaon/Gurugram           4707
Chennai                    3778
Pune                       3520
Noida                      2611
Vadodara                    719
Name: count, dtype: int64
Number of unique locations after cleaning: 10
Number of missing locations after cleaning: 0


### 3.7- Exploring rating column

In [12]:
# convert into numeric
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

# Calculate the median of 'rating'
median_rating = df['rating'].median()

# Fill missing values with the median rating
df['rating'].fillna(median_rating, inplace=True)
print(f"Number of missing ratings after cleaning: {df['rating'].isnull().sum()}")


Number of missing ratings after cleaning: 0


###3.8- Exploring reviews column

In [13]:
df['reviews'] = df['reviews'].astype(str)

# Remove text 'reviews' and keep only the numeric part
df['reviews'] = df['reviews'].str.replace('Reviews', '', regex=False).str.strip()

# Convert the 'reviews' column to numeric, forcing errors to NaN
df['reviews'] = pd.to_numeric(df['reviews'], errors='coerce')

# Calculate the median of the 'reviews' column
median_reviews = df['reviews'].median()

# Replace NaN values with the median
df['reviews'].fillna(median_reviews, inplace=True)

print(df['reviews'])


0        39110.0
1        32129.0
2        32129.0
3        32129.0
4         3759.0
          ...   
73762      231.0
73763      231.0
73764      231.0
73765        3.0
73766      231.0
Name: reviews, Length: 71738, dtype: float64


###3.9- Exploring responsibilities column

In [14]:
# Data Preprocessing
df['responsibilities'] = df['responsibilities'].str.lower()

# Handle Missing Values
missing_responsibilities = df['responsibilities'].isnull().sum()
print(f"Number of missing responsibilities: {missing_responsibilities}")

if missing_responsibilities > 0:
    df['responsibilities'].fillna('Not specified', inplace=True)
    print("Missing responsibilities filled with 'Not specified'.")

# Text Normalization
df['responsibilities'] = df['responsibilities'].str.replace('[^a-zA-Z\s]', '', regex=True)


# After Changes
print("\nFirst few rows of the cleaned 'responsibilities' column:")
print(df['responsibilities'].head())

Number of missing responsibilities: 0

First few rows of the cleaned 'responsibilities' column:
0         customer servicesalesrelationship management
1    product managementmarket analysischange manage...
2    business processchange managementteam manageme...
3    business processchange managementteam manageme...
4    sap bopl  sqloracle sqlsap business objects  x...
Name: responsibilities, dtype: object


###3.10- Exploring rest of the columns

In [15]:
# Drop column posted_on
df.drop(columns =['posted_on','job_link','company_link'], inplace=True)

# After the changes
print("Columns after deletion:")
print(df.columns)


Columns after deletion:
Index(['job_id', 'job_role', 'company', 'experience', 'salary', 'location',
       'rating', 'reviews', 'responsibilities', 'experience_range',
       'salary_range', 'location_cluster'],
      dtype='object')


##4- Information

In [16]:
# Create a copy of the DataFrame
df_copy = df.copy()

# Drop the 'experience' column and replace 'experience_range' with 'experience'
if 'experience' in df_copy.columns:
    df_copy.drop(columns=['experience'], inplace=True)

if 'experience_range' in df_copy.columns:
    df_copy.rename(columns={'experience_range': 'experience'}, inplace=True)

# Drop the 'salary' column and replace 'salary_range' with 'salary'
if 'salary' in df_copy.columns:
    df_copy.drop(columns=['salary'], inplace=True)

if 'salary_range' in df_copy.columns:
    df_copy.rename(columns={'salary_range': 'salary'}, inplace=True)

# Display the DataFrame
print("\nUpdated DataFrame:")
print(df_copy)

# Save the updated DataFrame to a local CSV file
output_file_path = 'updated_one.csv'
df_copy.to_csv(output_file_path, index=False)

print(f"The updated DataFrame has been saved to {output_file_path}")


Updated DataFrame:
               job_id                                           job_role  \
0       70123006070.0      Branch Banking - Calling For Women Candidates   
1       60123905908.0                       Product Owner Senior Manager   
2       60123905898.0  Employee Relations And Policies Associate Manager   
3       60123905897.0         Employee Relations And Policies Specialist   
4       60123008332.0                                  Sap Bo Consultant   
...               ...                                                ...   
73762   20123002989.0            Partner Success Executive/Edtech/Punjab   
73763   20123002957.0            Partner Success Associate/Edtech/Punjab   
73764  231222003986.0  Hiring For International Voice Process | Gurga...   
73765  171220001449.0                                   Fresher Engineer   
73766   71022008732.0            Anatomy Trainer - Part Time / Full Time   

                     company          location  rating  reviews  \
